# Consultando uma LLM

In [1]:
import llm, os, dotenv

dotenv.load_dotenv()

mistral_token = os.getenv("MISTRAL")
# mistral_token = <INSIRA SUA CHAVE AQUI>

modelo = llm.get_model("mistral-medium")

resposta = modelo.prompt(
    """
    Classifique o texto abaixo. Responda apenas com a classe, sem explicações.

    Texto: O Botafogo ganhou do PSG no Mundial.

    Classes: Política, Esportes, Economia, Outros.
    """,
    key=mistral_token
)

print(resposta)

Esportes


# Classificando um dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("celsowm/bbc_news_ptbr")

/home/abitporu/Documents/abraji/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = dataset['train'].to_pandas()
df.head()

,categoria,titulo,texto,data,link
0,brasil,"Quem foi Oswaldo Aranha, o brasileiro que ajud...",“Sua atuação no cenário internacional estava d...,2023-11-04,https://www.bbc.com/portuguese/articles/c0wxwr...
1,brasil,Como PCC fez Paraguai virar um dos países com ...,"No ranking de 2023, a nação saltou para a 4ª p...",2023-11-04,https://www.bbc.com/portuguese/articles/cp646z...
2,brasil,"Vídeo, Massa ou Milei: o que Brasil tem a ganh...",O segundo turno das eleições presidenciais na ...,2023-11-02,https://www.bbc.com/portuguese/articles/ckv0gk...
3,brasil,"Dia de Finados: como celebração dos mortos, qu...","""Do Senhor é a terra e a sua plenitude, o mund...",2023-11-02,https://www.bbc.com/portuguese/articles/c1354g...
4,brasil,A estratégia do Brasil por resolução sobre Isr...,Após quatro propostas terem sido rejeitadas na...,2023-10-31,https://www.bbc.com/portuguese/articles/clw7wl...


In [4]:
df.categoria.value_counts()

categoria
brasil            960
internacional     960
economia          960
saude             960
ciencia           960
tecnologia        960
america_latina    959
cultura           959
sociedade         959
Name: count, dtype: int64

In [5]:
classes = df.categoria.unique().tolist()
amostra = df.sample(n=20,random_state=42)

In [6]:
def classificar_texto(model, texto, classes):
    prompt = f"""
    Classifique o texto abaixo. Responda apenas com a classe, sem explicações.

    Texto: {texto}

    Classes: {classes}
    """
    
    resposta = str(model.prompt(prompt))   
    
    return resposta

classificar_texto(modelo, "O turismo no Chile cresceu", classes)

ModelError: 429: service_tier_capacity_exceeded - Service tier capacity exceeded for this model.

In [ ]:
from tqdm import tqdm
tqdm.pandas()

amostra['categoria_pred'] = amostra['categoria'].progress_apply(lambda x: classificar_texto(modelo, x, classes))

100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


In [ ]:
amostra

,categoria,titulo,texto,data,link,categoria_pred
1313,america_latina,5 sinais de recuperação — com limites — da eco...,"""Venezuela foi consertada"" é uma frase que cir...",2022-06-21,https://www.bbc.com/portuguese/internacional-6...,america_latina
7867,sociedade,'Meus pais iam me matar para limpar a honra da...,Quando Manoj — que foi registrado como sendo d...,2023-07-28,https://www.bbc.com/portuguese/articles/c25gpy...,sociedade
4595,saude,Vacina contra varíola dos macacos chega neste ...,O Ministério da Saúde anunciou que as primeira...,2022-09-19,https://www.bbc.com/portuguese/brasil-62317822,saude
1046,america_latina,O papel dos cartéis mexicanos na onda de violê...,"""Aqui pagamos com a vida pela democracia."" Seu...",2023-08-10,https://www.bbc.com/portuguese/articles/cglyke...,america_latina
3142,economia,Como crise do Credit Suisse manchou reputação ...,Segundo maior banco suíço e uma das 30 institu...,2023-03-21,https://www.bbc.com/portuguese/articles/crg02l...,economia
1672,america_latina,Como a Lei de Cotas incentivou a migração de a...,Criada em 2012 e prestes a passar por processo...,2020-12-29,https://www.bbc.com/portuguese/brasil-55225891,america_latina
6525,tecnologia,'Deus invisível': quem é o hacker acusado de r...,Só se sabia de três coisas sobre ele: 1) Fxmsp...,2020-07-31,https://www.bbc.com/portuguese/internacional-5...,tecnologia
5148,ciencia,Por que Nova York está afundando,O solo sob a cidade de Nova York está afundand...,2023-05-25,https://www.bbc.com/portuguese/articles/cq53nd...,ciencia
1905,america_latina,'Amazônia é como a bolsa de valores: dependend...,"Para o procurador Daniel Azeredo, do Ministéri...",2020-10-09,https://www.bbc.com/portuguese/brasil-54459038,america_latina
6959,cultura,Oscar 2023: a polêmica que pode desclassificar...,A indicação inesperada ao Oscar da atriz britâ...,2023-01-30,https://www.bbc.com/portuguese/geral-64460473,cultura


In [ ]:
# classifcation report
from sklearn.metrics import classification_report

print(classification_report(amostra.categoria, amostra.categoria_pred))


                precision    recall  f1-score   support

america_latina       1.00      1.00      1.00         4
        brasil       1.00      1.00      1.00         1
       ciencia       1.00      1.00      1.00         2
       cultura       1.00      1.00      1.00         1
      economia       1.00      1.00      1.00         2
 internacional       1.00      1.00      1.00         3
         saude       1.00      1.00      1.00         1
     sociedade       1.00      1.00      1.00         4
    tecnologia       1.00      1.00      1.00         2

      accuracy                           1.00        20
     macro avg       1.00      1.00      1.00        20
  weighted avg       1.00      1.00      1.00        20



# Classificando um dataset mais complicado

In [ ]:
def classificar_pl(model, texto):
    prompt = f"""
    Classifique se o PL é favorável ou contrário ao direito das mulheres. Responda apenas com 1 ou 0, sem explicações, de acordo com as definições:

    1: significa que o PL é desfavorável ao direito das mulheres.
    0: significa que o PL é favorável ao direito das mulheres.  

    ---

    Texto: {texto}

    Classes: 1, 0
    """
    
    resposta = str(model.prompt(prompt))   
    
    return resposta

classificar_pl(modelo,"Acabar com a Lei Maria da Penha")

'1'

In [ ]:
classificar_pl(modelo,"Apoiar vítimas de violência doméstica")

'0'

In [ ]:
ementas = load_dataset("azmina/ementas_anotadas_inteiroteor")

ementas_test = ementas['test'].to_pandas()

In [ ]:
ementas_test.head()

,id,text,label_tema,label_desfavoravel,proponente_mulher,partido,urlInteiroTeor,textoInteiroTeor
0,PL 4546/2023,Proíbe o ensino de determinados conteúdos rela...,genero,1,M,MDB,https://www.camara.leg.br/proposicoesWeb/prop_...,"PROJETO DE LEI Nº , DE 2023\n(DO SR. OLIV..."
1,PDL 92/2023,"Susta os efeitos da Portaria nº 665, de 23 dem...",genero,1,M,REPUBLICANOS,https://www.camara.leg.br/proposicoesWeb/prop_...,CÂMARA DOS DEPUTADOS\nGabinete do Deputado Mes...
2,PL 2092/2021,"Altera a Lei nº 8.080, de 19 de setembro de 19...",direitos sexuais e reprodutivos,0,M,PP,https://www.camara.leg.br/proposicoesWeb/prop_...,"\nPROJETO DE LEI Nº , DE 20 21 \n(Do Sr. ..."
3,PL 2736/2023,Estabelece a obrigatoriedade de submissão à mo...,violencia contra a mulher,1,M,PP,https://www.camara.leg.br/proposicoesWeb/prop_...,CÂMARA DOS DEPUTADOS \nGabinete do Deputado L...
4,PL 741/2021,"Altera o Decreto-Lei nº 2.848, de 7 de dezembr...",violencia contra a mulher,0,F,PROS,https://www.camara.leg.br/proposicoesWeb/prop_...,PROJETO DE LEI Nº 2021\n(Das Sras. Marga...


In [ ]:
amostra_ementas = ementas_test.sample(20,random_state=222)


amostra_ementas['label_pred'] = amostra_ementas['text'].progress_apply(lambda x: classificar_pl(modelo,x))

 51%|█████     | 51/100 [00:54<00:52,  1.07s/it]


ModelError: 429: invalid_request_error - Service tier capacity exceeded for this model.

In [ ]:
print(classification_report(amostra_ementas['label_desfavoravel'].astype(str), amostra_ementas['label_pred'].astype(str)))


              precision    recall  f1-score   support

           0       0.84      0.95      0.89        38
           1       0.71      0.42      0.53        12

    accuracy                           0.82        50
   macro avg       0.78      0.68      0.71        50
weighted avg       0.81      0.82      0.80        50

